In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import time
pd.set_option('html', False)

Introduction to Pandas
===

Data Structures
---

###Series
A Series is a one-dimensional object similar to an array, list, or column in a table. It will assign a labeled index to each item in the Series. By default, each item will receive an index label from 0 to N, where N is the length of the Series minus one.

In [ ]:
s = pd.Series([7, 'Heisenberg', 3.14, -1789710578, 'Happy Eating!'])
s

Alternatively, you can specify an index to use when creating the Series.

In [ ]:
s = pd.Series([7, 'Heisenberg', 3.14, -1789710578, 'Happy Eating!'],
              index=['A', 'Z', 'C', 'Y', 'E'])
s

The Series constructor can convert a dictonary as well, using the keys of the dictionary as its index.

In [ ]:
d = {'Chicago': 1000, 'New York': 1300, 'Portland': 900, 'San Francisco': 1100,
     'Austin': 450, 'Boston': None}
cities = pd.Series(d)
cities

You can use the index (or list of indices) to select specific items from the Series

In [ ]:
cities['Chicago']


In [ ]:
cities[['Chicago', 'Portland', 'San Francisco']]

Or you can use boolean indexing for selection.

In [ ]:
cities[cities < 1000]


Mathematical operations can be done using scalars and functions.

In [ ]:
cities ** 2

In [ ]:
np.log(cities)

###DataFrame

A DataFrame is a tablular data structure comprised of rows and columns, akin to a spreadsheet, database table, or R's data.frame object. You can also think of a DataFrame as a group of Series objects that share an index (the column names).

In [ ]:
data = {'year': [2010, 2011, 2012, 2011, 2012, 2010, 2011, 2012],
        'team': ['Liverpool', 'Liverpool', 'Liverpool', 'Chelsea', 'Chelsea', 'Everton', 'Everton', 'Everton'],
        'wins': [11, 8, 10, 15, 11, 6, 10, 4],
        'losses': [5, 8, 6, 1, 5, 10, 6, 12]}
football = pd.DataFrame(data, columns=['year', 'team', 'wins', 'losses'])
print football

A new column can be added to the dataframe with the use of a series object, the length of the series doesn't have to match that of the dataframe

In [ ]:
month = pd.Series(['Jan', 'Feb', 'Mar', 'Apr']*2)
football['month'] = month
football

However, when using other forms of data structures, such as arrays or lists, the length has to match the number of columns.

In [ ]:
time_of_day = ['AM', 'PM']
football['time'] = time_of_day

The different columns of the dataframe can be accessed using the column names either like a dictionary or as a varible within the dataframe

In [ ]:
football['team']

The individual rows can be gotten by accessing the index variable 'ix' with the index of the row

In [ ]:
football.ix[0]

Reading from File
---

In [ ]:
temp = '%s.csv'
path = temp % 'AAPL'
!wc -l $path

In [ ]:
aapl_bars = pd.read_csv(temp % 'AAPL')
aapl_bars.head()

Time series operations
---

In [ ]:
aapl_bars.dt

In [ ]:
aapl_bars.index = pd.to_datetime(aapl_bars.pop('dt'))

In [ ]:
aapl_bars.head()

In [ ]:
def load_bars(ticker):
    bars = pd.read_csv(temp % ticker)
    bars.index = pd.to_datetime(bars.pop('dt'))
    return bars

In [ ]:
aapl_bars.at_time(time(14, 30)).head(10)

In [ ]:
aapl_bars.close_price['2015-03-19']

Data alignment
---

In [ ]:
ts1 = pd.Series(np.random.randn(10), 
                index=pd.date_range('1/1/2000', periods=10))
ts1

In [ ]:
ts2 = ts1[[0, 2, 4, 5, 6, 7, 8]]
ts2

In [ ]:
ts1 + ts2

In [ ]:
df = pd.DataFrame({'A': ts1, 'B': ts2})
df

In [ ]:
ibm_bars = load_bars('IBM')

In [ ]:
def subsample(frame, pct=0.9):
    N = len(frame)
    indexer = np.sort(np.random.permutation(N)[:pct*N])
    return frame.take(indexer)

f1 = subsample(ibm_bars)
f2 = subsample(aapl_bars)
f1

In [ ]:
both = pd.concat([f1, f2], axis=1, keys=['IBM', 'AAPL'])
both.head(20)

Missing data handling
---

In [ ]:
df

In [ ]:
df.count()

In [ ]:
both.count()

In [ ]:
df.sum()

In [ ]:
df.mean(1)

In [ ]:
df.dropna()

In [ ]:
df.fillna(0)

In [ ]:
df.fillna(method='ffill')

In [ ]:
df.asfreq('4h')

In [ ]:
df.asfreq('4h').ffill(limit=3)

Data Analysis & Plots
---

In [ ]:
aapl_bars2 = load_bars('AAPL2')
mth_mean = aapl_bars2.close_price.resample('Q', how=['mean', 'median', 'std'])
mth_mean

In [ ]:
mth_mean.plot()

In [ ]:
mth_mean.groupby(mth_mean.index.month).aggregate(np.mean).plot(kind='bar')

Regression Analysis
---

In [ ]:
import scipy.stats as stats
x = np.array(range(1, 101))
y = 2*x + np.random.normal(20, 1000, size = 100)

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
print "y = ", slope, "x + ", intercept
print "R squared ", r_value**2

In [ ]:
plt.scatter(x, y, label = 'Data')
plt.plot((slope * x) + intercept, 'r-', label='Predicted')
plt.legend(loc='upper left');

In [ ]:
y = x**2 + np.random.normal(2, 10, size = 100) ** 2
z = np.polyfit(x, y, 4)
x, y

In [ ]:
plt.scatter(x, y)
px = np.poly1d(z)
plt.plot(x, px(x), 'r-')

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(np.log(x), np.log(y))
print slope, r_value**2

Groupby operations
---

In [ ]:
import random, string
import matplotlib as mpl
def rands(n):
    choices = string.ascii_letters
    return ''.join([random.choice(choices) for _ in xrange(n)])
mpl.rc('figure', figsize=(12, 8))

ind_names = np.array(['ENERGY', 'FINANCIAL', 'TECH', 
                      'CONSDUR', 'SERVICES', 'UTILITIES'], dtype='O')
ccys = np.array(['USD', 'EUR'], dtype='O')

Nfull = 2000
tickers = np.array(sorted(rands(5).upper() for _ in xrange(Nfull)), dtype='O')
tickers = np.unique(tickers)

industries = pd.Series(ind_names.take(np.random.randint(0, 6, Nfull)), 
                       index=tickers, name='industry')
ccy = pd.Series(ccys.take(np.random.randint(0, len(ccys), Nfull)), 
                index=tickers, name='ccy')

In [ ]:
ccy

In [ ]:
df = pd.DataFrame({'Momentum' : np.random.randn(1000) / 200 + 0.03,
                'Value' : np.random.randn(1000) / 200 + 0.08,
                'ShortInterest' : np.random.randn(1000) / 200 - 0.02},
                index=tickers.take(np.random.permutation(Nfull)[:1000]))
df.head()

In [ ]:
means = df.groupby(industries).mean()
means

In [ ]:
means.plot(kind='barh')

In [ ]:
means = df.groupby([industries, ccy]).mean()
means

In [ ]:
keys = [industries, ccy]
zscore = lambda x: (x - x.mean()) / x.std()
normed = df.groupby(keys).apply(zscore)

In [ ]:
normed.groupby(keys).agg(['mean', 'std'])

Hierarchical indexing
---

In [ ]:
means

In [ ]:
means['Momentum']

In [ ]:
means.ix['TECH']

In [ ]:
means.stack()

In [ ]:
means.stack().unstack('industry')

Merging and joining
---

In [ ]:
base = '/Users/wesm/Dropbox/book/svn/book_scripts/movielens/ml-1m'
get_path = lambda x: '%s/%s.dat' % (base, x)

unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table(get_path('users'), sep='::', header=None, names=unames)

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(get_path('ratings'), sep='::', header=None, names=rnames)
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(get_path('movies'), sep='::', header=None, names=mnames)

In [ ]:
movies.head()

In [ ]:
ratings.head()

In [ ]:
users.head()

In [ ]:
data = pd.merge(pd.merge(ratings, users), movies)
data

In [ ]:
rating_counts = data.groupby('title').size()
freq_titles = rating_counts.index[rating_counts > 1000]
freq_titles

In [ ]:
highest_rated = data.groupby('title').rating.mean()[freq_titles].order()[-20:]
highest_rated

In [ ]:
filtered = data[data.title.isin(highest_rated.index)]
filtered.title = filtered.title.str[:25]
filtered.groupby(['title', 'gender']).rating.count().unstack()

Pivot tables
---

In [ ]:
mean_ratings = data.pivot_table('rating', rows='title',
                                cols='gender', aggfunc='mean')
mean_ratings.tail(20)

Data summary, statistics
---
summary, value_counts, etc.

In [ ]:
data.title.value_counts()

In [ ]:
data.rating.describe()

In [ ]:
by_gender = data.groupby('gender').rating.describe()
by_gender

In [ ]:
by_gender.unstack(0)